In [1]:
#from ib_insync import *
import numpy as np
import pandas as pd
import math
from collections import deque
from os import listdir
from os.path import isfile, join
from helpers import *

In [2]:
mypath = 'eric_jh_data/'
countries = sorted(['Australia', 'Japan', 'China'])
fx_dict = {'Australia':('eric_jh_data/Forex/AUD_USD_new.csv',0),
           'Japan':('eric_jh_data/Forex/USD_JPY_new.csv',1),
           'China':('eric_jh_data/Forex/USD_HKD_new.csv',1)}

list_pairs = []
for country in countries:
    countrypath = mypath + country
    adr_names =  [f for f in listdir(countrypath) if not isfile(join(countrypath, f))] #grab all adr names of the country
    for adr in sorted(adr_names):
        list_pairs.append((country, adr))

In [3]:
# date - date in local time
# All dates are in local time: so in sequential order (for each row), it will go stock_open, stock_close, adr_open, adr_close
# avg_bid_non_us_before - how much foreign currency we can buy with 1 USD, 1 minute before the Asian market opens
# avg_ask_non_us_before - how much foreign currency we would need to sell for 1 USD, 1 minute before the Asian market opens
# avg_non_us_before - average of avg_bid_non_us_before and avg_ask_non_us_before
# avg_bid_non_us_at - how much foreign currency we can buy with 1 USD, when the Asian market opens
# avg_ask_non_us_at - how much foreign currency we would need to sell for 1 USD, when the Asian market opens
# avg_non_us_at - average of avg_bid_non_us_at and avg_ask_non_us_at
# avg_bid_us_before - how much foreign currency we can buy with 1 USD, 1 minute before the US market opens
# avg_ask_us_before - how much foreign currency we would need to sell for 1 USD, 1 minute before the US market opens
# avg_us_before - average of avg_bid_us_before and avg_ask_us_before
# avg_bid_us_at - how much foreign currency we can buy with 1 USD, when the US market opens
# avg_ask_us_at - how much foreign currency we would need to sell for 1 USD, when the US market opens
# avg_us_at - average of avg_bid_us_at and avg_ask_us_at
# ir - foreign interest rate
# stock_num_per_unit - how many stocks we would buy for 1 "unit" of trade
# adr_num_per_unit - how many adr shares we would sell for 1 "unit" of trade

merged_df = data_processing(*list_pairs[1], fx_dict)
merged_df.head()

,date,adr_open,adr_close,adr_volume,stock_open,stock_close,stock_volume,avg_bid_non_us_before,avg_ask_non_us_before,avg_non_us_before,...,avg_bid_us_at,avg_ask_us_at,avg_us_at,ir,stock_num_per_unit,adr_num_per_unit,stock_open_per_unit,stock_close_per_unit,adr_open_per_unit,adr_close_per_unit
0,2015-04-10,18.08,17.36,1873,0.040,0.039,987003,1.296948,1.299612,1.298280,...,1.302409,1.305068,1.303738,NaN,600.0,1,24.0,23.4,18.08,17.36
1,2015-04-13,17.76,16.16,2762,0.039,0.039,1586945,1.303310,1.305987,1.304648,...,1.315530,1.318215,1.316872,NaN,600.0,1,23.4,23.4,17.76,16.16
2,2015-04-14,16.96,17.08,2545,0.036,0.038,2905099,1.316900,1.319606,1.318253,...,1.309820,1.312529,1.311174,NaN,600.0,1,21.6,22.8,16.96,17.08
3,2015-04-15,17.68,17.52,2106,0.039,0.038,2069419,1.312108,1.314822,1.313465,...,1.310301,1.312994,1.311647,NaN,600.0,1,23.4,22.8,17.68,17.52
4,2015-04-16,17.40,17.00,1068,0.038,0.036,2378678,1.301459,1.304133,1.302796,...,1.282035,1.284634,1.283335,NaN,600.0,1,22.8,21.6,17.40,17.00


In [4]:
list_pairs_copy = list_pairs.copy()
list_pairs_aus = list_pairs[:9]
list_pairs_chi = list_pairs[9:18]
list_pairs_jap = list_pairs[18:]

In [5]:
# Pairs we have chosen to trade
list_pairs = [("Australia", "PLL_PLL"),
               ("Australia", "MESO_MSB"),
               ("Australia", "GENE_GTG"),
               ("Australia", "WBK_WBC"),
               ("Australia", "KZIA_KZA"),
               ("Australia", "IMMP_IMM"),
               ("Australia", "IMRN_IMC"),
               ("Australia", "ATHE_ATH"),
               ("Japan", "SONY_6758"), 
               ("Japan", "TAK_4502"),
               ("Japan", "TM_7203"),
               ("Japan", "SMFG_8316"),
               ("China", "BGNE_6160"), 
               ("China", "SNP_386")]

In [6]:
fname = 'logs/results1_sfx_all.txt'
with open(fname, 'r') as f:
    is_res = f.readlines()
hp = {}
for i in range(len(list_pairs_copy)):
    hp[list_pairs_copy[i]] = [float(x) for x in is_res[i*5 + 4].split("(")[1].split(")")[0].split(", ")]

# Final Strategy

In [7]:
df_dict = {}
for (country, adr) in list_pairs_copy:
    df_dict[(country, adr)] = data_processing(country, adr, fx_dict)
    
date_set = set()
for (country, adr) in list_pairs_copy:
    date_set = date_set.union(set(df_dict[(country, adr)]["date"]))
datelist = sorted(date_set)

In [8]:
def final_strategy(df_dict, datelist, trading_limits, allocation, hp, list_pairs, cash = 250000, 
                   start_date = "2016-01-01", end_date = "2021-01-31", slippage_bps = 10, borrowing_bps = 50, 
                   risk_lookback = 100, cooldown = 100, cooldown_limit = 100, var_ci = 0.95, var_limit = 0.1, 
                   max_drawdown_limit = 0.2, sigma_limit = 0.05, stop_loss_limit = 0.15, maximum_holding_period = 30, 
                   volume_lookback = 5, adjust_for_risk = False):

    # Number of pairs from each country we will trade at each time
    num_traded = {"Australia" : [], "China" : [], "Japan" : []}

    hp_dict = {}
    for (country, adr) in list_pairs:
        hp_dict[(country, adr)] = {}
        hp_dict[(country, adr)]["lookback"] = int(hp[(country, adr)][0])
        hp_dict[(country, adr)]["entry"] = hp[(country, adr)][1]
        hp_dict[(country, adr)]["exit"] = hp[(country, adr)][2]
        hp_dict[(country, adr)]["actual_entries"] = []
        hp_dict[(country, adr)]["actual_exits"] = []
        hp_dict[(country, adr)]["stop_loss"] = hp[(country, adr)][3]
        # Fraction of cash allocated to each adr-stock pair
        hp_dict[(country, adr)]["allocation"] = allocation[country]/trading_limits[country]
        hp_dict[(country, adr)]["original_allocation"] = allocation[country]/trading_limits[country]

    diff_record_dict = {}
    for (country, adr) in list_pairs:
        diff_record_dict[(country, adr)] = deque(maxlen = 2*hp_dict[(country, adr)]["lookback"])

    conditions = {}
    for (country, adr) in list_pairs:
        conditions[(country, adr)] = {}
        conditions[(country, adr)]["enter_cond1"] = False
        conditions[(country, adr)]["exit_cond1"] = False
        conditions[(country, adr)]["enter_cond2"] = False
        conditions[(country, adr)]["exit_cond2"] = False

    iter_dict = {}
    for (country, adr) in list_pairs:
        iter_dict[(country, adr)] = {}
        iter_dict[(country, adr)]["index"] = 0

    positions = {}
    for (country, adr) in list_pairs:
        positions[(country, adr)] = {}
        positions[(country, adr)]["stock_pos"] = 0
        positions[(country, adr)]["adr_pos"] = 0
        positions[(country, adr)]["prev_adr_pos"] = 0
        positions[(country, adr)]["holding_period"] = None
        positions[(country, adr)]["trade_type"] = None

    forex_dict = {}
    for (country, adr) in list_pairs:
        forex_dict[(country, adr)] = {}
        forex_dict[(country, adr)]["forex_cash"] = 0
        forex_dict[(country, adr)]["prev_forex_value"] = 0

    trade_records = []
    portfolio_values = []
    portfolio_values_for_risk = []
    dates = []
    # Determine if risk limits have been breached
    breach1 = False
    breach2 = False
    
    # Accounts for slippage and transaction costs
    short_multiplier = 1 - 0.0001*slippage_bps
    long_multiplier = 1 + 0.0001*slippage_bps
    starting_cash = cash
    # For book-keeping, since we shall store the portfolio value of the day before
    prev_cash = cash
    
    for i in range(1, len(datelist)):
        current_date = datelist[i]
        prev_date = datelist[i-1]
        if current_date >= start_date and prev_date <= end_date:
            # Log number of stocks traded in each country
            # Log portfolio value of the day before, when the Asian market opens
            cooldown += 1
            dates.append(prev_date)
            prev_value = prev_cash
            for country in countries:
                num_traded[country].append(0)
            for (country, adr) in list_pairs:
                index = iter_dict[(country, adr)]["index"]
                merged_df = df_dict[(country, adr)]
                row = merged_df.loc[index]
                if positions[(country, adr)]["stock_pos"] > 0:
                    num_traded[country][-1] += 1
                if index > 0:
                    # Add adr values
                    prev_value += positions[(country, adr)]["prev_adr_pos"]*merged_df.loc[index - 1, 'adr_close']
                    # Add stock values
                    forex_dict[(country, adr)]["prev_forex_value"] = forex_dict[(country, adr)]["forex_cash"] 
                    forex_dict[(country, adr)]["prev_forex_value"] += positions[(country, adr)]["stock_pos"]*row['stock_open']
                    if forex_dict[(country, adr)]["prev_forex_value"] > 0:
                        # For our convention, ask price is the price of selling foreign currency (how much foreign currency to sell for 1 USD)
                        # Ask price > Bid price
                        forex_dict[(country, adr)]["prev_forex_value"] /= row['avg_ask_non_us_at']
                    elif forex_dict[(country, adr)]["prev_forex_value"] < 0:
                        forex_dict[(country, adr)]["prev_forex_value"] /= row['avg_bid_non_us_at']
                    prev_value += forex_dict[(country, adr)]["prev_forex_value"]
            portfolio_values.append(prev_value)
            portfolio_values_for_risk.append(prev_value)

            # Account for borrowing cost
            for (country, adr) in list_pairs:
                index = iter_dict[(country, adr)]["index"]
                merged_df = df_dict[(country, adr)]
                positions[(country, adr)]["prev_adr_pos"] = positions[(country, adr)]["adr_pos"]
                if positions[(country, adr)]["stock_pos"] > 0:
                    positions[(country, adr)]["holding_period"] += 1
                    # Cost of shorting stock
                    cash -= 0.0001*borrowing_bps*(1/252)*abs(positions[(country, adr)]["adr_pos"])*merged_df.loc[index - 1, 'adr_close']
                    # Cost of shorting cash
                    multiplier = (1 + 0.01*(2 + merged_df.loc[index]["ir"])*(1/252))
                    forex_dict[(country, adr)]["forex_cash"] *= multiplier
            prev_cash = cash

            # Risk Tests
            if cooldown >= cooldown_limit and adjust_for_risk:
                max_drawdown = calc_max_drawdown(portfolio_values_for_risk, method = "absolute")
                starting_cash = portfolio_values_for_risk[0]
                cum_pl = prev_value - starting_cash
                # Fraction to liquidate
                liquidate_frac = 0
                # If max drawdown / cum PnL hits 50% or 75% of limit
                # Liquidate a fraction of positions and reduce the amount traded for future transactions
                # If max drawdown / cum PnL hits 100% of limit
                # Liquidate all positions, stop trading for {cooldown} days, reset all risk measures
                if max_drawdown > max_drawdown_limit*starting_cash or cum_pl < -stop_loss_limit*starting_cash:
                    liquidate_frac = 1
                    cooldown = 0
                    portfolio_values_for_risk = []
                    breach1 = False
                    breach2 = False
                    # Return to original allocation after cooldown.
                    # In real-life, someone would step in and redecide if this is appropriate
                    for (country, adr) in list_pairs:
                        hp_dict[(country, adr)]["allocation"] = hp_dict[(country, adr)]["original_allocation"]
                elif (max_drawdown > 0.75*max_drawdown_limit*starting_cash or cum_pl < -0.75*stop_loss_limit*starting_cash) and not breach2:
                    liquidate_frac = 0.5
                    for (country, adr) in list_pairs:
                        hp_dict[(country, adr)]["allocation"] *= (1/3)
                    breach2 = True
                elif (max_drawdown > 0.5*max_drawdown_limit*starting_cash or cum_pl < -0.5*stop_loss_limit*starting_cash) and not breach1:
                    liquidate_frac = 1/3
                    for (country, adr) in list_pairs:
                        hp_dict[(country, adr)]["allocation"] *= (1/2)
                    breach1 = True

                # Calculate VaR and PnL volatility for each prior day at Asian market open
                # Further adjust fraction to liquidate based on these risk limits
                stock_values = np.zeros(risk_lookback)
                adr_values = np.zeros(risk_lookback)
                for (country, adr) in list_pairs:
                    index = iter_dict[(country, adr)]["index"]
                    merged_df = df_dict[(country, adr)]
                    if index > 0:
                        temp_risk_lookback = min(risk_lookback, index)
                        current = merged_df.loc[(index - temp_risk_lookback + 1):index].copy()
                        ind_stock_values = (np.array(current["stock_open"])/np.array(current["avg_non_us_at"]))*positions[(country, adr)]["stock_pos"] 
                        ind_adr_values = np.array(merged_df.loc[(index - temp_risk_lookback):(index-1)]["adr_close"]*positions[(country, adr)]["adr_pos"])
                        stock_values[-len(ind_stock_values):] += ind_stock_values
                        adr_values[-len(ind_adr_values):] += ind_adr_values
                sigma, var, max_drawdown_abs = get_risk_statistics(stock_values, adr_values, var_ci)
                if sigma > prev_value*sigma_limit:
                    new_liquidate_frac = (sigma - prev_value*sigma_limit)/(prev_value*sigma_limit)
                    liquidate_frac = max(liquidate_frac, new_liquidate_frac)

                if var > prev_value*var_limit:
                    new_liquidate_frac = (var - prev_value*var_limit)/(prev_value*var_limit)
                    liquidate_frac = max(liquidate_frac, new_liquidate_frac)

                # Liquidate if necessary
                if liquidate_frac > 0:
                    for (country, adr) in list_pairs:
                        index = iter_dict[(country, adr)]["index"]
                        merged_df = df_dict[(country, adr)]
                        if index > 0 and index+1 < len(merged_df):
                            row = merged_df.loc[index]
                            stock_pos = positions[(country, adr)]["stock_pos"]
                            adr_pos = positions[(country, adr)]["adr_pos"]
                            if stock_pos > 0:
                                units_traded = stock_pos/row["stock_num_per_unit"]
                                units_liquidated = math.ceil(units_traded*liquidate_frac)
                                stock_quantity_sold = int(units_liquidated*row["stock_num_per_unit"])
                                adr_quantity_bought = int(units_liquidated*row["adr_num_per_unit"])
                                adr_px = row['adr_open']*long_multiplier
                                cash -= adr_quantity_bought*adr_px
                                prev_cash -= adr_quantity_bought*adr_px

                                stock_px_fx = merged_df.loc[index+1,'stock_open']*short_multiplier
                                leftover_forex_cash = forex_dict[(country, adr)]["forex_cash"]*liquidate_frac
                                leftover_forex_cash += stock_px_fx*stock_quantity_sold
                                forex_dict[(country, adr)]["forex_cash"] *= (1-liquidate_frac)
                                if leftover_forex_cash > 0:
                                    leftover_forex_cash /= merged_df.loc[index+1,'avg_ask_non_us_at']
                                else:
                                    leftover_forex_cash /= merged_df.loc[index+1,'avg_bid_non_us_at']
                                cash += leftover_forex_cash
                                prev_cash += leftover_forex_cash

                                trade_records.append("Liquidating positions:\n")
                                # Times in EST
                                trade_records.append(f"We bought {-adr_pos} shares of ADR ({adr_name}) at the price of {adr_px} on {row['date']}\n")
                                trade_records.append(f"We sold {stock_pos} shares of underlying stock ({stock_name}) at the price of {stock_px_fx} foreign dollars on {row['date']}\n")

                                positions[(country, adr)]["adr_pos"] += adr_quantity_bought
                                positions[(country, adr)]["stock_pos"] -= stock_quantity_sold
                                positions[(country, adr)]["prev_adr_pos"] = positions[(country, adr)]["adr_pos"]
                                if liquidate_frac == 1:
                                    positions[(country, adr)]["holding_period"] = None
                                    num_traded[country][-1] += 1

        for (country, adr) in list_pairs:
            index = iter_dict[(country, adr)]["index"]
            merged_df = df_dict[(country, adr)]
            # Lookback parameter is in number of trading days
            lookback = 2*hp_dict[(country, adr)]["lookback"]

            if index+1 < len(merged_df):
                row = merged_df.loc[index]
                if row["date"] == current_date:
                    iter_dict[(country, adr)]["index"] += 1
                    if index > 0: 
                        diff_record = diff_record_dict[(country, adr)]
                        entry = hp_dict[(country, adr)]["entry"]
                        exit = hp_dict[(country, adr)]["exit"]
                        stop_loss = hp_dict[(country, adr)]["stop_loss"]
                        stock_pos = positions[(country, adr)]["stock_pos"]
                        adr_pos = positions[(country, adr)]["adr_pos"]
                        holding_period = positions[(country, adr)]["holding_period"]
                        trade_type = positions[(country, adr)]["trade_type"]
                        stock_name = adr.split("_")[1]
                        adr_name = adr.split("_")[0]
                        enter_cond1 = conditions[(country, adr)]["enter_cond1"]
                        exit_cond1 = conditions[(country, adr)]["exit_cond1"]
                        enter_cond2 = conditions[(country, adr)]["enter_cond2"]
                        exit_cond2 = conditions[(country, adr)]["exit_cond2"]

                        # Before the US Market open, append price difference between ADR and stock
                        diff_record.append(merged_df.loc[index-1,'adr_close_per_unit'] - row['stock_close_per_unit']/row['avg_us_before'])
                        if len(diff_record) == lookback and row["date"] >= start_date and row["date"] <= end_date and cooldown >= cooldown_limit:
                            mean = np.array(diff_record).mean()
                            std = np.array(diff_record).std()
                            
                            # Check that a concurrent trade was not already placed
                            if not (enter_cond2 or exit_cond2):
                                enter_cond1 = (diff_record[-1] > mean + entry*std
                                               and diff_record[-1] <= mean + stop_loss*std
                                               and stock_pos == 0 and adr_pos == 0 
                                               and num_traded[country][-1] < trading_limits[country])
                                exit_cond1 = ((diff_record[-1] < mean + exit*std
                                              or diff_record[-1] > mean + stop_loss*std
                                              or (holding_period == maximum_holding_period and trade_type == 1))
                                              and stock_pos > 0 and adr_pos < 0)

                                if enter_cond1:
                                    portfolio_value_before_entering = portfolio_values[-1] if portfolio_values else cash
                                    # Allow ourselves to trade 20% of ADT volume over the past 5 trading days
                                    # We take the median to make this estimate more robust to extreme values
                                    adr_volume = 0.2*(merged_df.loc[index-volume_lookback:index - 1,:]["adr_volume"].median()/row["adr_num_per_unit"])
                                    stock_volume = 0.2*(merged_df.loc[index-volume_lookback+1:index,:]["stock_volume"].median()/row["stock_num_per_unit"])
                                    units = int(min((hp_dict[(country, adr)]["allocation"]*cash)/merged_df.loc[index-1,'adr_close_per_unit'],
                                                    (hp_dict[(country, adr)]["allocation"]*cash)/(row['stock_close_per_unit']/merged_df.loc[index+1,'avg_us_before']), 
                                                    adr_volume, 
                                                    stock_volume))
                                    adr_quantity = int(units*row["adr_num_per_unit"])
                                    stock_quantity = int(units*row["stock_num_per_unit"])

                                    # Take portfolio value for each previous day when the US market opens
                                    # Further adjust volume based on historical max drawdown, VaR and PnL volatility
                                    temp_risk_lookback = min(risk_lookback, index)
                                    current = merged_df.loc[(index - temp_risk_lookback + 1):index].copy()
                                    stock_values = np.array((current["stock_close"]/current["avg_us_before"])*stock_quantity) 
                                    adr_values = np.array(merged_df.loc[(index - temp_risk_lookback):(index-1)]["adr_close"]*adr_quantity)
                                    sigma, var, max_drawdown_abs = get_risk_statistics(stock_values, adr_values, var_ci)
                                    if (var > portfolio_value_before_entering*var_limit or 
                                        max_drawdown_abs > max_drawdown_limit*starting_cash or 
                                        sigma > portfolio_value_before_entering*sigma_limit):
                                        frac = min((portfolio_value_before_entering*var_limit)/var, 
                                                   (max_drawdown_limit*starting_cash)/max_drawdown_abs,
                                                  (portfolio_value_before_entering*sigma_limit)/sigma)
                                        units = int(frac*units)
                                        if units == 0:
                                            enter_cond1 = False
                                        adr_quantity = int(units*row["adr_num_per_unit"])
                                        stock_quantity = int(units*row["stock_num_per_unit"]) 
                                        
                                    if units != 0:
                                        # Short ADR
                                        adr_pos -= adr_quantity
                                        positions[(country, adr)]["adr_pos"] -= adr_quantity
                                        adr_px = row['adr_open']*short_multiplier
                                        cash += adr_quantity*adr_px
                                        prev_cash += adr_quantity*adr_px
                                        actual_price_difference = row['adr_open_per_unit'] - row['stock_close_per_unit']/row['avg_us_before']
                                        actual_entry = (actual_price_difference - mean)/std
                                        hp_dict[(country, adr)]["actual_entries"].append(actual_entry)
                                        
                                        # Long Stock
                                        stock_pos += stock_quantity
                                        positions[(country, adr)]["stock_pos"] += stock_quantity
                                        stock_px_fx = merged_df.loc[index+1,'stock_open']*long_multiplier
                                        forex_dict[(country, adr)]["forex_cash"] -= stock_px_fx*stock_quantity

                                        positions[(country, adr)]["prev_adr_pos"] = positions[(country, adr)]["adr_pos"]
                                        positions[(country, adr)]["holding_period"] = 0
                                        positions[(country, adr)]["trade_type"] = 1
                                        num_traded[country][-1] += 1

                                        trade_records.append("Opening positions:\n")
                                        # Times in EST
                                        trade_records.append(f"We sold {adr_quantity} shares of ADR ({adr_name}) at the price of {adr_px} on {row['date']}\n")
                                        trade_records.append(f"We bought {stock_quantity} shares of underlying stock ({stock_name}) at the price of {stock_px_fx} foreign dollars on {row['date']}\n")

                                elif exit_cond1:
                                    # Long ADR
                                    adr_px = row['adr_open']*long_multiplier
                                    cash -= abs(adr_pos)*adr_px
                                    prev_cash -= abs(adr_pos)*adr_px
                                    actual_price_difference = row['adr_open_per_unit'] - row['stock_close_per_unit']/row['avg_us_before']
                                    actual_exit = (actual_price_difference - mean)/std
                                    hp_dict[(country, adr)]["actual_exits"].append(actual_exit)
                                    
                                    # Short Stock
                                    stock_px_fx = merged_df.loc[index+1,'stock_open']*short_multiplier
                                    forex_dict[(country, adr)]["forex_cash"] += stock_px_fx*stock_pos
                                    if forex_dict[(country, adr)]["forex_cash"] > 0:
                                        forex_dict[(country, adr)]["forex_cash"] /= merged_df.loc[index+1,'avg_ask_non_us_at']
                                    else:
                                        forex_dict[(country, adr)]["forex_cash"] /= merged_df.loc[index+1,'avg_bid_non_us_at']
                                    cash += forex_dict[(country, adr)]["forex_cash"]
                                    prev_cash += forex_dict[(country, adr)]["forex_cash"]
                                    forex_dict[(country, adr)]["forex_cash"] = 0

                                    trade_records.append("Closing positions:\n")
                                    # Times in EST
                                    trade_records.append(f"We bought {-adr_pos} shares of ADR ({adr_name}) at the price of {adr_px} on {row['date']}\n")
                                    trade_records.append(f"We sold {stock_pos} shares of underlying stock ({stock_name}) at the price of {stock_px_fx} foreign dollars on {row['date']}\n")

                                    positions[(country, adr)]["stock_pos"] = 0
                                    positions[(country, adr)]["adr_pos"] = 0
                                    positions[(country, adr)]["holding_period"] = None
                                    positions[(country, adr)]["trade_type"] = None
                                    positions[(country, adr)]["prev_adr_pos"] = positions[(country, adr)]["adr_pos"]
                                    num_traded[country][-1] -= 1

                                conditions[(country, adr)]["enter_cond1"] = enter_cond1
                                conditions[(country, adr)]["exit_cond1"] = exit_cond1

                        # Before the Asian Market open, append price difference between ADR and stock
                        diff_record.append(row['adr_close_per_unit'] - row['stock_close_per_unit']/merged_df.loc[index+1,'avg_non_us_before'])
                        if len(diff_record) == lookback and row["date"] >= start_date and merged_df.loc[index+1,"date"] <= end_date:
                            mean = np.array(diff_record).mean()
                            std = np.array(diff_record).std()

                            # Check that a concurrent trade was not already placed
                            if not (enter_cond1 or exit_cond1):
                                enter_cond2 = (diff_record[-1] > mean + entry*std
                                               and diff_record[-1] <= mean + stop_loss*std
                                               and stock_pos == 0 and adr_pos == 0 
                                               and num_traded[country][-1] < trading_limits[country])
                                exit_cond2 = ((diff_record[-1] < mean + exit*std
                                              or diff_record[-1] > mean + stop_loss*std
                                              or (holding_period == maximum_holding_period and trade_type == 2))
                                              and stock_pos > 0 and adr_pos < 0)

                                if enter_cond2:
                                    # Allow ourselves to trade 20% of ADT volume over the past 5 trading days
                                    # We take the median to make this estimate more robust to extreme values
                                    portfolio_value_before_entering = portfolio_values[-1] if portfolio_values else cash
                                    adr_volume = 0.2*(merged_df.loc[index-volume_lookback+1:index,:]["adr_volume"].median()/row["adr_num_per_unit"])
                                    stock_volume = 0.2*(merged_df.loc[index-volume_lookback+1:index,:]["stock_volume"].median()/row["stock_num_per_unit"])
                                    units = int(min((hp_dict[(country, adr)]["allocation"]*cash)/row['adr_close_per_unit'],
                                                    (hp_dict[(country, adr)]["allocation"]*cash)/(row['stock_close_per_unit']/merged_df.loc[index+1,'avg_non_us_before']), 
                                                    adr_volume, 
                                                    stock_volume))
                                    adr_quantity = int(units*row["adr_num_per_unit"])
                                    stock_quantity = int(units*row["stock_num_per_unit"])

                                    # Take portfolio value for each previous day when the Asian market opens
                                    # Further adjust volume based on historical Max Drawdown, VaR and PnL volatility
                                    temp_risk_lookback = min(risk_lookback, index)
                                    current = merged_df.loc[(index - temp_risk_lookback + 1):index].copy()
                                    next_day = merged_df.loc[(index - temp_risk_lookback + 2):(index + 1)].copy()
                                    stock_values = (np.array((current["stock_close"])/np.array(next_day["avg_non_us_before"]))*stock_quantity) 
                                    adr_values = np.array(current["adr_close"]*adr_quantity)
                                    sigma, var, max_drawdown_abs = get_risk_statistics(stock_values, adr_values, var_ci)
                                    if (var > portfolio_value_before_entering*var_limit or 
                                        max_drawdown_abs > max_drawdown_limit*starting_cash or 
                                        sigma > portfolio_value_before_entering*sigma_limit):
                                        frac = min((portfolio_value_before_entering*var_limit)/var, 
                                                   (max_drawdown_limit*starting_cash)/max_drawdown_abs,
                                                  (portfolio_value_before_entering*sigma_limit)/sigma)
                                        units = int(frac*units)
                                        if units == 0:
                                            enter_cond2 = False
                                        adr_quantity = int(units*row["adr_num_per_unit"])
                                        stock_quantity = int(units*row["stock_num_per_unit"])   
                                    if units != 0:
                                        # Long stock
                                        stock_pos += stock_quantity
                                        positions[(country, adr)]["stock_pos"] += stock_quantity
                                        stock_px_fx = merged_df.loc[index+1,'stock_open']*long_multiplier
                                        forex_dict[(country, adr)]["forex_cash"] -= stock_px_fx*stock_quantity
                                        # We store the current cash/adr position for book-keeping purposes, 
                                        # because the trade below will occur on the next day (EST)
                                        positions[(country, adr)]["prev_adr_pos"] = positions[(country, adr)]["adr_pos"]
                                        
                                        actual_price_difference = row['adr_close_per_unit'] - merged_df.loc[index+1,'stock_open_per_unit']/merged_df.loc[index+1,'avg_non_us_before']
                                        actual_entry = (actual_price_difference - mean)/std
                                        hp_dict[(country, adr)]["actual_entries"].append(actual_entry)
                                        
                                        # Short ADR
                                        adr_pos -= adr_quantity
                                        positions[(country, adr)]["adr_pos"] -= adr_quantity
                                        adr_px = merged_df.loc[index+1,'adr_open']*short_multiplier
                                        cash += adr_quantity*adr_px

                                        positions[(country, adr)]["holding_period"] = 0
                                        positions[(country, adr)]["trade_type"] = 2
                                        num_traded[country][-1] += 1
                                        trade_records.append("Opening positions:\n")
                                        # Times in EST
                                        trade_records.append(f"We bought {stock_quantity} shares of underlying stock ({stock_name}) at the price of {stock_px_fx} foreign dollars on {row['date']}\n")
                                        trade_records.append(f"We sold {adr_quantity} shares of ADR ({adr_name}) at the price of {adr_px} on {merged_df.loc[index+1,'date']}\n")

                                # Liquidation condition
                                elif exit_cond2:
                                    # Short stock
                                    stock_px_fx = merged_df.loc[index+1,'stock_open']*short_multiplier
                                    forex_dict[(country, adr)]["forex_cash"] += stock_px_fx*stock_pos
                                    if forex_dict[(country, adr)]["forex_cash"] > 0:
                                        forex_dict[(country, adr)]["forex_cash"] /= merged_df.loc[index+1,'avg_ask_non_us_at']
                                    else:
                                        forex_dict[(country, adr)]["forex_cash"] /= merged_df.loc[index+1,'avg_bid_non_us_at']
                                    cash += forex_dict[(country, adr)]["forex_cash"]
                                    prev_cash += forex_dict[(country, adr)]["forex_cash"]
                                    forex_dict[(country, adr)]["forex_cash"] = 0
                                    # We store the current cash/adr position for book-keeping purposes, 
                                    # because the trade below will occur on the next day (EST)
                                    positions[(country, adr)]["prev_adr_pos"] = positions[(country, adr)]["adr_pos"]
                                    
                                    actual_price_difference = row['adr_close_per_unit'] - merged_df.loc[index+1,'stock_open_per_unit']/merged_df.loc[index+1,'avg_non_us_before']
                                    actual_exit = (actual_price_difference - mean)/std
                                    hp_dict[(country, adr)]["actual_exits"].append(actual_exit)
                                    
                                    # Long ADR
                                    adr_px = merged_df.loc[index+1,'adr_open']*long_multiplier
                                    cash -= abs(adr_pos)*adr_px
                                    trade_records.append("Closing positions:\n")
                                    # Times in EST
                                    trade_records.append(f"We sold {stock_pos} shares of underlying stock ({stock_name}) at the price of {stock_px_fx} foreign dollars on {row['date']}\n")
                                    trade_records.append(f"We bought {-adr_pos} shares of ADR ({adr_name}) at the price of {adr_px} on {merged_df.loc[index+1,'date']}\n")

                                    positions[(country, adr)]["stock_pos"] = 0
                                    positions[(country, adr)]["adr_pos"] = 0
                                    positions[(country, adr)]["holding_period"] = None
                                    positions[(country, adr)]["trade_type"] = None
                                    num_traded[country][-1] -= 1

                                conditions[(country, adr)]["enter_cond2"] = enter_cond2
                                conditions[(country, adr)]["exit_cond2"] = exit_cond2

    return dates, portfolio_values, trade_records, hp_dict

In [9]:
trading_limits = {"Australia" : 4, "China" : 1, "Japan" : 3}
allocation = {"Australia" : 0.3, "China" : 0.45, "Japan" : 0.25}
dates, portfolio_values, trade_records, hp_dict = final_strategy(df_dict, datelist, trading_limits, allocation, hp, list_pairs, start_date = "2016-01-01", end_date = "2021-01-31")
ret = np.round(100*(portfolio_values[-1] - portfolio_values[0])/portfolio_values[0], 2)
print("Returns: {}%".format(ret))

Returns: 64.86%


In [10]:
df = pd.DataFrame(columns = ["country", "adr", "hp_entry", "actual_entry", "hp_exit", "actual_exit"])
for key in hp_dict:
    row = [*key, hp_dict[key]["entry"], np.mean(hp_dict[key]["actual_entries"]), hp_dict[key]["exit"], np.mean(hp_dict[key]["actual_exits"])]
    df.loc[len(df)] = row
df.to_csv("actual_hps.csv")